# Driver Decomposition
Given a list of drivers, decompose them into objects containing:
1. Concept
2. Polarity (for now sense)
3. Adjective

In [ ]:
import json
import spacy

In [ ]:
nlp = spacy.load('en_core_web_lg')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
from spacy.symbols import nsubj, VERB, NOUN, PROPN
from spacy import displacy

In [ ]:
drivers = [line.rstrip('\n') for line in open('../drivers.txt')]
nounCountList = {}

In [ ]:
drivers

In [ ]:
def cleanDriver(driver):
    return (' '.join(filter(lambda x: x.lower() not in spacy_stopwords,  driver.split())))

In [ ]:
def genConceptJson(driver):
    
    #driver = cleanDriver(driver)
    
    res = {}
    res["concepts"] = {}
    res["concepts"]["count"] = 0
    i = 1

    for token in nlp(driver):

        if (token.pos in [NOUN, PROPN]):
            
            nounCountList[token.lemma_] = 1 if token.lemma_ not in nounCountList else nounCountList[token.lemma_] + 1
                
            res["concepts"]["count"] += 1
            res["concepts"]["concept_"+str(i)] = {}
            res["concepts"]["concept_"+str(i)]["lemma"] = token.lemma_

            if (len([child.lemma_ for child in token.children])):
                res["concepts"]["concept_"+str(i)]["sense"] = [child.lemma_ for child in token.children if (child.pos_ in ['VERB','ADJ','NOUN'])]

            i += 1 

    for token in nlp(driver):
        if (token.pos == VERB):
            children = [child.lemma_ for child in token.children]
            #print (children)

            if (len(children) > 0):
                for keys in res["concepts"]:
                    if keys != "count":

                        for child in children:
                            #print ("RES",res["concepts"][keys]["lemma"])
                            
                            if (res["concepts"][keys]["lemma"] == child):
                                res["concepts"][keys]["adj"] = token.lemma_
                                

    return (res)

In [ ]:
sDrivers = ['government open to foreign Information Technology support, regulations in place']

In [ ]:
conceptJson = {}
for driver in drivers:
    conceptJson[driver] = genConceptJson(driver)
    
#print(conceptJson)

In [ ]:
print (json.dumps(conceptJson, indent=3))

In [ ]:
nounCountList

In [ ]:
sorted_nounList = sorted(nounCountList.items(), key=lambda kv: kv[1])

In [ ]:
sorted_nounList

## Observations

#### 1. Minor changes in the drivers can lead to major changes in the driver decomposition outupt.

In [ ]:
doc = nlp("businesses struggling to keep up with clients demands")

In [ ]:
displacy.render(doc, style='dep')

In [ ]:
doc = nlp("businesses struggling to keep up with client's demands")

In [ ]:
displacy.render(doc, style='dep')

 #### 2. Stopword removal is helpful in some cases but not in others.

In [ ]:
doc = nlp("government open to foreign Information Technology support, regulations in place")

In [ ]:
displacy.render(doc, style='dep')